In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML
import msgpack as mp
import msgpack_numpy as mpn
import base64
from datetime import datetime
import os
from tqdm import tqdm
from datetime import timedelta
import keyboard
import time

In [2]:
# base_pth = "C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 08-02-22_13-24-38_725"
# base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 04-02-22_14-03-00_359"
# base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 10-02-22_09-14-15_436"
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 17-02-22_13-45-40_738" # first exp on hc 05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 17-02-22_17-13-29_526" # second exp on hc05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_11-02-02_575" # third with datetime corrected exp on hc05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_12-12-01_66" # fourth
base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_13-03-19_712" # fifth

_prm = os.listdir(base_pth)

for i in _prm:
    if i.startswith("PARAMS"):
        _p_fname = i
prm_file = base_pth + "\\" + _p_fname
prm_file


p = open(prm_file, "rb")
unpacker = mp.Unpacker(p, object_hook=mpn.decode)
prm = []
for unpacked in unpacker:
    prm.append(unpacked)
prm.pop(0)
prm.pop(0)


video_time = []
for i in prm:
    video_time.append(datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S.%f'))
v_start = video_time[0]
v_start


"""for df"""
video_time_df = []
for i in prm:
    video_time_df.append(i[0])



In [8]:

"""frame by frame"""
video_data = cv2.VideoCapture(base_pth+"//Video.avi")

frame_no = 0
while video_data.isOpened:
    ret = True
    video_data.set(1, frame_no)

    if video_data:
        temp = video_data.read()

    frame = temp[1]
    # print(frame)
    
    if not ret:
        break
    if keyboard.is_pressed("a"):
        frame_no+=1
        # print(frame_no)
    
    if keyboard.is_pressed("d"):
        frame_no-=1
    frame = cv2.flip(frame, 1)

    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # org
    org = (50, 50)
    
    # fontScale
    fontScale = 1
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.putText() method
    image = cv2.putText(frame, str(video_time[frame_no])[10:], org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
    if ret:
        cv2.imshow("",frame)
        # plt.show()
        
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    
    
    
    
cv2.destroyAllWindows()
        

In [4]:
# plt.plot(imu_1["imu_time"], imu_1["ax"])
# imu_2 = datetime_to_s(imu_2, "sys_time", v_start)

"""for df"""

imu_1df = pd.read_csv(base_pth+"//imu01.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_1df.index.name = "time"
# imu_2df = pd.read_csv(base_pth+"//2_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
# imu_2df.index.name = "time"
vid_df = pd.DataFrame(video_time_df, columns=["time"])
vid_df = pd.to_datetime(vid_df["time"])

# imu_1df = imu_1df.rename(columns={'ax.1': 'az'})
imu_1df

,gx,gy,gz,ax,ay,az
time,,,,,,
2022-02-18 13:03:36.060240,0.018289,0.059847,-0.030717,-0.059496,0.299166,1.051833
2022-02-18 13:03:36.196563,0.155694,0.014046,-0.015450,-0.056811,0.295016,1.050124
2022-02-18 13:03:36.197563,0.201495,-0.062290,-0.015450,-0.057543,0.291354,1.048415
2022-02-18 13:03:36.197563,0.094625,-0.031756,-0.030717,-0.054369,0.295016,1.052077
2022-02-18 13:03:36.197563,-0.012245,-0.062290,-0.000183,-0.053393,0.290865,1.045485
...,...,...,...,...,...,...
2022-02-18 13:04:02.661442,-0.149650,0.105649,0.030352,-0.059496,0.289400,1.048415
2022-02-18 13:04:02.661442,-0.073314,-0.062290,0.015085,-0.056322,0.291598,1.048903
2022-02-18 13:04:02.661442,-0.256520,0.014046,-0.015450,-0.056566,0.290133,1.048659


In [105]:
# plt.plot(imu_1["sys_sec"], imu_1["az"])
# imu_1df
vid_df

0     2022-02-18 13:03:29.286814
1     2022-02-18 13:03:29.354435
2     2022-02-18 13:03:29.420310
3     2022-02-18 13:03:29.487527
4     2022-02-18 13:03:29.555044
                 ...            
507   2022-02-18 13:04:03.085520
508   2022-02-18 13:04:03.153857
509   2022-02-18 13:04:03.219161
510   2022-02-18 13:04:03.285972
511   2022-02-18 13:04:03.353281
Name: time, Length: 512, dtype: datetime64[ns]

In [109]:
imu_1df.index.unique()[imu_1df.index.unique().get_loc(t, method='nearest')]

Timestamp('2022-02-18 13:03:38.222704')

In [6]:
# imu_1df.index.get_loc("2022-02-04 14:04:38.745642", method="nearest")

# imu_1df.index[0]
%matplotlib qt

In [11]:
plt.plot(imu_1df["ax"], label = "ax")
plt.plot(imu_1df["ay"], label = "ay")
plt.plot(imu_1df["az"], label = "az")
plt.ylim(-3,3)
plt.legend()


In [ ]:
# plt.plot(t, dat.ax)

In [ ]:
plt.plot(imu_1df["imu_time"].diff())